In [1]:
import gc
import os
import talib
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timezone, time


import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short
from config import ccxt_connect, prices_collection, binance_client
from helper import vwap, current_position

load_dotenv()

client = ccxt_connect()
binance = binance_client()
prices = prices_collection()

In [3]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f"Finding the last record in our DB: \n{last_date.strftime('%m-%d-%y-%H:%M:%S')}")

print(f'Querying data from bybit to make current')
new_data_query = binance.get_historical_klines('ETHUSDT', '5m', str(last_date), limit=1000)
print('Getting new data from Bybit API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print(f'Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'

signal = df['signal'].iloc[-1]
price = df['close'].iloc[-1]
position = current_position()
print(f'Current signal: {signal}, current position: {position}')

Finding the last record in our DB: 
12-26-22-15:35:00
Querying data from bybit to make current
Getting new data from Bybit API (5m interval)
Inserting new data into DB
Calculating signals
Current signal: None, current position: SELL


In [4]:
df.tail(40)

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
260,63a9c01a6e93764f764a07b8,1219.45,1219.45,1219.25,1219.45,2.288200e+04,46.960641,-0.021044,1219.536034,1219.535913,1216.680245,2022-12-26 12:50:00.000,None
261,63a9c01a6e93764f764a07b9,1219.45,1219.45,1219.20,1219.20,2.023200e+04,44.073771,-0.033905,1219.468827,1219.523916,1216.705317,2022-12-26 12:55:00.000,SELL
262,63a9c01a6e93764f764a07ba,1219.20,1219.25,1219.20,1219.25,1.303400e+04,44.804592,-0.036115,1219.425062,1219.514134,1216.730637,2022-12-26 13:00:00.000,SELL
263,63a9c01a6e93764f764a07bb,1219.25,1219.25,1218.20,1218.35,5.273400e+04,35.749008,-0.092078,1219.210049,1219.472557,1216.746750,2022-12-26 13:05:00.000,SELL
264,63a9c01a6e93764f764a07bc,1218.35,1218.65,1211.50,1216.55,4.739157e+06,24.906646,-0.236025,1218.678039,1219.368180,1216.744793,2022-12-26 13:10:00.000,SELL
265,63a9c01a6e93764f764a07bd,1216.55,1217.05,1216.20,1216.85,8.786100e+04,28.783454,-0.293017,1218.312432,1219.278245,1216.745839,2022-12-26 13:15:00.000,SELL
266,63a9c01a6e93764f764a07be,1216.85,1217.00,1216.30,1216.95,4.234540e+05,30.079266,-0.304615,1218.039945,1219.195094,1216.747871,2022-12-26 13:20:00.000,SELL
267,63a9c01a6e93764f764a07bf,1216.95,1217.50,1216.90,1217.15,1.431680e+05,32.716125,-0.280078,1217.861956,1219.122055,1216.751872,2022-12-26 13:25:00.000,SELL
268,63a9c01a6e93764f764a07c0,1217.15,1217.50,1217.00,1217.25,1.566030e+05,34.055234,-0.239921,1217.739565,1219.055196,1216.756829,2022-12-26 13:30:00.000,SELL
269,63a9c01a6e93764f764a07c1,1217.25,1217.45,1215.55,1215.90,1.116700e+05,26.412716,-0.284069,1217.371652,1218.942510,1216.748303,2022-12-26 13:35:00.000,SELL


In [82]:
orders = client.fetch_closed_orders(symbol)

In [84]:
orders[-1]['info']['side']

'Sell'

In [86]:
int(orders[-1]['info']['qty'])

2798

In [ ]:
if signal == position or signal == None:
        print('No change in position, sleep for 5min and rescan')

else:
        with open('current_position.txt', 'w') as f:
                f.write(str(signal))

if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))